Helpful Links:
Geomapping: https://stackoverflow.com/questions/65064137/geopandas-how-to-plot-countries-cities

In [ ]:
#import virtualenvironment
wd = os.getcwd()
wd_venv = wd +"/Virtualenvironment"
os.chdir(wd_venv)

def pip_install():
    with cd(wd_venv):
        with prefix('source venv/bin/activate'):
            run('pip install -r requirements.txt')
os.chdir(wd)

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt  
import matplotlib.animation 
#import matplotlib
#matplotlib.use('TkAgg')
import numpy as np
#Wetter-API Import
import requests 

import os
import pandas as pd
import psycopg2
import contextily as ctx
from shapely.geometry import Point
from matplotlib.animation import FuncAnimation
from IPython import display
from IPython.display import HTML
from celluloid import Camera
from mpl_toolkits.axes_grid1 import make_axes_locatable


: 

In [3]:
#read files inside folder for translational data


from typing import List
import pathlib
wd = os.getcwd()
wd = wd+"/Prepared_Data"
def find_excel_files_in(directory:pathlib.Path) -> List[pathlib.Path]:
    
    files:List[pathlib.Path] = list()
    for filepath in directory.rglob('*.csv'):
        if filepath.is_file():
            files.append(filepath)

    return files

# List of your directories
directories:List[str] = [format(wd)]

found_files:List[pathlib.Path] = list()

for directory in directories:
    directory:pathlib.Path = pathlib.Path(directory)
    found_files.extend(find_excel_files_in(directory))

#read excel files
import pandas as pd

#print(files)
    #Deleting all empty rows in the file df
for files in found_files:
    data = pd.read_csv(files)


In [4]:
data.optimized_assignee.unique()

NameError: name 'data' is not defined

In [ ]:
data["country"].unique()

In [ ]:
#make countrynames lowercase
data["country"] = data["country"].str.lower()

In [ ]:
#hong kong to china in order to be displayed in worldmap later
data["country"] = data.apply(lambda x: "china" if x["country"]== 'hong kong (s.a.r.)' else x["country"],axis =1)

In [ ]:
data[data["country"]=="china"]

In [ ]:
data[["publication_number","country"]]

In [ ]:
#data = data[data["country"]!="china"]

In [ ]:
#DELETE THIS SHIT
from pandas import NA


percentage_column = data[data['country'].values != ""]
percentage_column = percentage_column[["publication_number","country"]].groupby('country').agg('count')*100.0/len(data)
percentage_column.rename(columns = {"publication_number" : "proportion_of_publication_number"}, inplace = True)
percentage_column.reset_index(inplace = True)
percentage_column["proportion_of_publication_number"] = percentage_column.apply(lambda x: NA if x["proportion_of_publication_number"] == 0 else x["proportion_of_publication_number"],axis = 1)
data_all = data.merge(percentage_column, left_on = 'country', right_on = 'country', how='left')
data_all

#create data for proportion and add it to data df BUT FOR GRANTED ONLY
prestep= data[data['country'].values != ""]
percentage_column = prestep[prestep["simplified_status"] == "granted"]
percentage_column = percentage_column[["publication_number","country"]].groupby('country').agg('count')*100.0/len(data)
percentage_column.rename(columns = {"publication_number" : "proportion_of_publication_number"}, inplace = True)
percentage_column.reset_index(inplace = True)
percentage_column["proportion_of_publication_number"] = percentage_column.apply(lambda x: NA if x["proportion_of_publication_number"] == 0 else x["proportion_of_publication_number"],axis = 1)
data_granted = data.merge(percentage_column, left_on = 'country', right_on = 'country', how='left')
data_granted

#create data for proportion and add it to data df BUT FOR NOT-GRANTED ONLY
prestep= data[data['country'].values != ""]
percentage_column = prestep[prestep["simplified_status"] == "not granted"]
percentage_column = percentage_column[["publication_number","country"]].groupby('country').agg('count')*100.0/len(data)
percentage_column.rename(columns = {"publication_number" : "proportion_of_publication_number"}, inplace = True)
percentage_column.reset_index(inplace = True)
percentage_column["proportion_of_publication_number"] = percentage_column.apply(lambda x: NA if x["proportion_of_publication_number"] == 0 else x["proportion_of_publication_number"],axis = 1)
data_not_granted = data.merge(percentage_column, left_on = 'country', right_on = 'country', how='left')
data_not_granted


In [ ]:
percentage_column.sort_values(by="proportion_of_publication_number", ascending = False)

In [ ]:
#get current working directory

current_wd = os.getcwd()
path_worldmap= "/Worldmap/World_Countries.shp"
path_worldmap = current_wd+path_worldmap
path_worldmap

In [ ]:
#read worldmap shape file
world_map = gpd.read_file(path_worldmap)

In [ ]:
world_map

In [ ]:
#make country names lowercase in order to match dataframes by countrynames
world_map["COUNTRY"]=world_map["COUNTRY"].str.lower()
world_map.sort_values(by = "COUNTRY")

In [ ]:
world_map[world_map["COUNTRY"]=="united states of america"]

In [ ]:
world_map.loc[[229]]

In [ ]:
data_all.columns

In [ ]:
#adjust important names for merging
world_map["COUNTRY"] = world_map["COUNTRY"].str.replace("united states", "united states of america")
world_map["COUNTRY"] = world_map["COUNTRY"].str.replace("south korea", "korea (south)")
world_map["COUNTRY"] = world_map["COUNTRY"].str.replace("moldova", "republic of moldova")
world_map["COUNTRY"] = world_map["COUNTRY"].str.replace("russia", "russian federation")
#data["country"] = data["country"].str.replace("hong kong (s.a.r.)", "china")
#world_map["COUNTRY"] = world_map["COUNTRY"].str.replace(f"hong kong (s.a.r)", "china")



In [ ]:
world_map[world_map["COUNTRY"]== "united states"]

In [ ]:
world_map

In [ ]:
#merge dataframes
merged_all = world_map.merge(data_all, left_on = 'COUNTRY', right_on = 'country', how='outer')
merged_all['proportion_of_publication_number'] = merged_all['proportion_of_publication_number'].fillna(0)

#merge granted
merged_granted = world_map.merge(data_granted, left_on = 'COUNTRY', right_on = 'country', how='outer')
merged_granted['proportion_of_publication_number'] = merged_granted['proportion_of_publication_number'].fillna(0)

merged_not_granted = world_map.merge(data_not_granted, left_on = 'COUNTRY', right_on = 'country', how='outer')
merged_not_granted['proportion_of_publication_number'] = merged_not_granted['proportion_of_publication_number'].fillna(0)


In [ ]:
#create list of countries that are not on the world map

countries_not_on_map = []
for element in list(percentage_column["country"]):
    if element not in list(world_map["COUNTRY"]):
        countries_not_on_map.append(element)
 
print(countries_not_on_map)

In [ ]:
#extract proportions for if "country" is no actual country
nocountries = percentage_column["country"].isin(countries_not_on_map)
nocountries = percentage_column[nocountries]
nocountries

In [ ]:
merged_all.info()

In [ ]:
merged_all.describe()

In [ ]:
#merged= merged[merged["COUNTRY"] != "china"]

In [ ]:
#plot worldmap ALL

import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm
from matplotlib.colors import TwoSlopeNorm
import statistics

#statistics.median(percentage_column["proportion_of_publication_number"])


fig, ax = plt.subplots(figsize = (20,10))

#divider = make_axes_locatable(ax)

#cax = divider.append_axes("right", size="5%", pad=0.1)

# create a normalized colorbar
norm = TwoSlopeNorm(vmin=min(percentage_column["proportion_of_publication_number"]),
                    vcenter=statistics.median(percentage_column["proportion_of_publication_number"]),
                    vmax=max(percentage_column["proportion_of_publication_number"]))
cmap=plt.cm.get_cmap('nipy_spectral', 50)
#cbar = plt.cm.ScalarMappable(norm=norm, cmap=cmap)

#creating values for tick in legend; 20 ticks
ticks = []
for x in range(1,21):
    ticks.append(round(x/20*max(merged_all["proportion_of_publication_number"]),1))

tick_label = []
for x in range(1,21):
    tick_label.append(f"{round(x/20*max(merged_all['proportion_of_publication_number']),1)}%")

#not in use yet
legend_label = []
for x in range(1,21):
    legend_label.append(f"{round(x/20*max(merged_all['proportion_of_publication_number']),1)}%")
legend_label

fig = merged_all.plot(ax = ax,markersize = 100,column = 'proportion_of_publication_number', figsize=(20,10),
                                                     cmap=cmap,
                                                     # create a normalized colorbar
                                                     #cbar = plt.cm.ScalarMappable(norm=norm),  
                                                     legend = True,
                                                    
                                                     legend_kwds={'label': 'Proportion of Patents in %',
                                                                    #'title_fontsize':20,
                                                                    'orientation': "horizontal",
                                                                    'pad': 0.04,
                                                                    'fraction': 0.09,
                                                                   # 'norm': norm,
                                                                    'ticks':ticks
                                                                    #"fmt":"{:%}"
                                                                    
                                                                    },
                                                     missing_kwds={
                                                                 "color":"grey",
                                                                 "edgecolor":"black",
                                                                 "hatch":"---",
                                                                 "label":"Missing Values"
                                                                },
                                                     vmin=0, vmax=max(merged_all["proportion_of_publication_number"])
                                                     )


#include no countries in the title
#creation of string
#preparation for no country entries
nocountries_string =f"{str(nocountries.iloc[0][0])}: {str(round(nocountries.iloc[0][1],2))}% \n "
nocountries_string
for i in range(len(nocountries)):
    if i == 0:
        continue
    nocountries_string = nocountries_string + f"{str(nocountries.iloc[i][0])}: {str(round(nocountries.iloc[i][1],2))}% \n "

#ax_cbar = fig.colorbar(cbar, ax=ax)
#plt.legend(title = "lol",loc = "lower center")
plt.title(f"Worldwide distribution of\nAll Patents", fontsize = 30, loc = "left",y = 1.01,style = 'italic')
from datetime import datetime
now = datetime.now()
plt.title(f"{now.strftime('%d/%m/%Y %H:%M')}", loc='right', fontsize=15, color='grey', style='italic')
plt.tick_params(left = False, bottom = False)
#ax.axes.set_xticklabels(tick_label)


ax.axes.get_xaxis().set_visible(False)
ax.axes.get_yaxis().set_visible(False)

plt.show()

In [ ]:
nocountries_string

GRANTED

In [ ]:
#plot worldmap GRANTED

import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm
from matplotlib.colors import TwoSlopeNorm
import statistics

#statistics.median(percentage_column["proportion_of_publication_number"])


fig, ax = plt.subplots(figsize = (20,10))

#divider = make_axes_locatable(ax)

#cax = divider.append_axes("right", size="5%", pad=0.1)

# create a normalized colorbar
#norm = TwoSlopeNorm(vmin=min(percentage_column["proportion_of_publication_number"]),
#                    vcenter=statistics.median(percentage_column["proportion_of_publication_number"]),
#                    vmax=max(percentage_column["proportion_of_publication_number"]))
cmap=plt.cm.get_cmap('nipy_spectral', 50)
#cbar = plt.cm.ScalarMappable(norm=norm, cmap=cmap)

#creating values for tick in legend; 20 ticks
ticks = []
for x in range(1,21):
    ticks.append(round(x/20*max(merged_granted["proportion_of_publication_number"]),1))

tick_label = []
for x in range(1,21):
    tick_label.append(f"{round(x/20*max(merged_granted['proportion_of_publication_number']),1)}%")

#not in use yet
legend_label = []
for x in range(1,21):
    legend_label.append(f"{round(x/20*max(merged_granted['proportion_of_publication_number']),1)}%")
legend_label

fig = merged_granted.plot(ax = ax,markersize = 100,column = 'proportion_of_publication_number', figsize=(20,10),
                                                     cmap=cmap,
                                                     # create a normalized colorbar
                                                     #cbar = plt.cm.ScalarMappable(norm=norm),  
                                                     legend = True,
                                                    
                                                     legend_kwds={'label': 'Proportion of Patents in %',
                                                                    #'title_fontsize':20,
                                                                    'orientation': "horizontal",
                                                                    'pad': 0.04,
                                                                    'fraction': 0.09,
                                                                   # 'norm': norm,
                                                                    'ticks':ticks
                                                                    #"fmt":"{:%}"
                                                                    
                                                                    },
                                                     missing_kwds={
                                                                 "color":"grey",
                                                                 "edgecolor":"black",
                                                                 "hatch":"---",
                                                                 "label":"Missing Values"
                                                                },
                                                     vmin=0, vmax=max(merged_granted["proportion_of_publication_number"])
                                                     )


#include no countries in the title
#creation of string
#preparation for no country entries
nocountries_string =f"{str(nocountries.iloc[0][0])}: {str(round(nocountries.iloc[0][1],2))}% \n "
nocountries_string
for i in range(len(nocountries)):
    if i == 0:
        continue
    nocountries_string = nocountries_string + f"{str(nocountries.iloc[i][0])}: {str(round(nocountries.iloc[i][1],2))}% \n "

#ax_cbar = fig.colorbar(cbar, ax=ax)
#plt.legend(title = "lol",loc = "lower center")
plt.title(f"Worldwide distribution of\nGranted Patents", fontsize = 30, loc = "left",y = 1.01,style = 'italic')
from datetime import datetime
now = datetime.now()
plt.title(f"{now.strftime('%d/%m/%Y %H:%M')}", loc='right', fontsize=15, color='grey', style='italic')
plt.tick_params(left = False, bottom = False)
#ax.axes.set_xticklabels(tick_label)


ax.axes.get_xaxis().set_visible(False)
ax.axes.get_yaxis().set_visible(False)

plt.show()

Not-Granted

In [ ]:
#plot worldmap NOT-GRANTED

import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm
from matplotlib.colors import TwoSlopeNorm
import statistics

#statistics.median(percentage_column["proportion_of_publication_number"])


fig, ax = plt.subplots(figsize = (20,10))

#divider = make_axes_locatable(ax)

#cax = divider.append_axes("right", size="5%", pad=0.1)

# create a normalized colorbar
#norm = TwoSlopeNorm(vmin=min(percentage_column["proportion_of_publication_number"]),
#                    vcenter=statistics.median(percentage_column["proportion_of_publication_number"]),
#                    vmax=max(percentage_column["proportion_of_publication_number"]))
cmap=plt.cm.get_cmap('nipy_spectral', 50)
#cbar = plt.cm.ScalarMappable(norm=norm, cmap=cmap)

#creating values for tick in legend; 20 ticks
ticks = []
for x in range(1,21):
    ticks.append(round(x/20*max(merged_not_granted["proportion_of_publication_number"]),1))

tick_label = []
for x in range(1,21):
    tick_label.append(f"{round(x/20*max(merged_not_granted['proportion_of_publication_number']),1)}%")

#not in use yet
legend_label = []
for x in range(1,21):
    legend_label.append(f"{round(x/20*max(merged_not_granted['proportion_of_publication_number']),1)}%")
legend_label

fig = merged_not_granted.plot(ax = ax,markersize = 100,column = 'proportion_of_publication_number', figsize=(20,10),
                                                     cmap=cmap,
                                                     # create a normalized colorbar
                                                     #cbar = plt.cm.ScalarMappable(norm=norm),  
                                                     legend = True,
                                                    
                                                     legend_kwds={'label': 'Proportion of Patents in %',
                                                                    #'title_fontsize':20,
                                                                    'orientation': "horizontal",
                                                                    'pad': 0.04,
                                                                    'fraction': 0.09,
                                                                   # 'norm': norm,
                                                                    'ticks':ticks
                                                                    #"fmt":"{:%}"
                                                                    
                                                                    },
                                                     missing_kwds={
                                                                 "color":"grey",
                                                                 "edgecolor":"black",
                                                                 "hatch":"---",
                                                                 "label":"Missing Values"
                                                                },
                                                     vmin=0, vmax=max(merged_not_granted["proportion_of_publication_number"])
                                                     )


#include no countries in the title
#creation of string
#preparation for no country entries
nocountries_string =f"{str(nocountries.iloc[0][0])}: {str(round(nocountries.iloc[0][1],2))}% \n "
nocountries_string
for i in range(len(nocountries)):
    if i == 0:
        continue
    nocountries_string = nocountries_string + f"{str(nocountries.iloc[i][0])}: {str(round(nocountries.iloc[i][1],2))}% \n "

#ax_cbar = fig.colorbar(cbar, ax=ax)
#plt.legend(title = "lol",loc = "lower center")
plt.title(f"Worldwide distribution of\nNot-Granted Patents", fontsize = 30, loc = "left",y = 1.01,style = 'italic')
from datetime import datetime
now = datetime.now()
plt.title(f"{now.strftime('%d/%m/%Y %H:%M')}", loc='right', fontsize=15, color='grey', style='italic')
plt.tick_params(left = False, bottom = False)
#ax.axes.set_xticklabels(tick_label)


ax.axes.get_xaxis().set_visible(False)
ax.axes.get_yaxis().set_visible(False)

plt.show()

In [ ]:
data.columns

In [ ]:
data

In [ ]:
pip freeze > requirements.txt

affine==2.3.1
appnope @ file:///Users/ktietz/demo/mc3/conda-bld/appnope_1629146036738/work
argon2-cffi @ file:///opt/conda/conda-bld/argon2-cffi_1645000214183/work
argon2-cffi-bindings @ file:///private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/croot-wbf5edig/argon2-cffi-bindings_1644845754377/work
asttokens @ file:///opt/conda/conda-bld/asttokens_1646925590279/work
attrs @ file:///opt/conda/conda-bld/attrs_1642510447205/work
backcall @ file:///home/ktietz/src/ci/backcall_1611930011877/work
beautifulsoup4 @ file:///private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_croot-15cbtalq/beautifulsoup4_1650462161715/work
bleach @ file:///opt/conda/conda-bld/bleach_1641577558959/work
bokeh==2.4.3
Bottleneck @ file:///private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_07078715-3ab7-4562-8d3d-d56b0eaa0f7dp504n_ny/croots/recipe/bottleneck_1657175566567/work
branca==0.5.0
bs4==0.0.1
celluloid==0.2.0
certifi @ file:///private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/ab

In [ ]:
merged_all

In [ ]:
import folium

map = folium.Map()
map

In [ ]:
!pip install plotly

In [ ]:
import plotly.express as px

df = px.data.gapminder().query("year==2007")
fig = px.choropleth(merged_all, locations="geometry",
                    color="proportion_of_publication_numbers", # lifeExp is a column of gapminder
                    hover_name="use", # column to add to hover information
                    color_continuous_scale=px.colors.sequential.Plasma)
fig.show()